In [1]:
# Random Forest Algorithm on Sonar Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import pandas as pd

In [2]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):   # n_folds = 5 given
    #print('len of data', len(dataset))
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)    
    #print('foldsize',fold_size)
    for i in range(n_folds):
        #print('i', i)
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            #print('index',index)
            fold.append(dataset_copy.pop(index))
            #print('len of dataset_copy after popping', len(dataset_copy))
            #print('fold',fold)
        dataset_split.append(fold)
        #print('datasplit',dataset_split)
        #print('len of datasplit',len(dataset_split))
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [3]:
#Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    #print('**********in evaluate algo*************')
    folds = cross_validation_split(dataset, n_folds)   # taking 5 cross validatoin data of containing 40 rows each from dataset 
    #print('crossfolds',len(folds))
    scores = list()
    for fold in folds:
        #print('fold',len(fold))    # iterating through each fold in 5 folds
        train_set = list(folds)
        train_set.remove(fold)  # remove that fold from train_set 
        train_set = sum(train_set, [])  #  sum(iterable, start) , iterable can be list tupple or dictionaries ,
                                        #  but most importantly it should be numbers. adding empty list to train_set list
        test_set = list()
        #print("train", len(train_set))
        
        for row in fold:       # iterating each row in that fold
            #print('row',row)
            row_copy = list(row)
            test_set.append(row_copy)  # add the rows to test_set from that fold
            row_copy[-1] = None
        #print("test", len(test_set)) 
        tree, predicted = algorithm(train_set, test_set, *args)
        #print('tree', tree)
        #print('predict', predicted)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return tree, scores


In [27]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    #print("********* in test_split******")
    #print('value', value)
    for row in dataset:
        #print('row[index]', row[index])
        #print('len of row', len(row))
        if row[index] < value:
            left.append(row)
            #print('left', left)
        else:
            right.append(row)
            #print('right', right)
    #print('left', len(left))
    #print('right', len(right))
    return left, right
 
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    #print("**************in gini_index**************")
    #print("n_instance", n_instances)   #  give 164.0 sum of (120 + 44)
    # sum weighted Gini index for each group
    gini = 0.0
    #print('group',groups)
    for group in groups:
        size = float(len(group))
        #print('len of group', size)  # will give the size of left and right groups i.e. 120.0(left) &  44.0(right) 
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            #print('class value', class_val)
            p = [row[-1] for row in group].count(class_val) / size  # counting frequency of the class values in the group
            #print('p', [row[-1] for row in group].count(class_val))
            score += p * p
        # weight the group score by its relative size
        #print('score',score)
        gini += (1.0 - score) * (size / n_instances)
    #print('gini', gini)
    return gini


# Select the best split point for a dataset
def get_split(dataset, n_features):
    #print("******** in get split **********")
    class_values = list(set(row[-1] for row in dataset))  # set the class values for last column of the row = {0,1}
    #print('calss values',class_values)
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:    # until len of feature < 7
        index = randrange(len(dataset[0])-1)  # generate the random index in range 60
        #print('index', index)
        #print('features', features)
        if index not in features:
            features.append(index)
    for index in features:    # iterate the index numbers(48,52,...) in feature i.e. features [48, 52, 54, 46, 14, 34]
        for row in dataset:
            groups = test_split(index, row[index], dataset)  # will give left and right group of that index column in row
            gini = gini_index(groups, class_values)  # will  give the gini index using left, right groups and class values
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}


# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)
 
# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):  # cross val trainset, max depth = 10, min size = 1, features= 7 
    root = get_split(train, n_features)  # return index, value and group
    split(root, max_depth, min_size, n_features, 1)
    #print("*****************************************************************************************")
    #print('root',root)
    return root
 
# Make a prediction with a decision tree
def predict(node, row):
    print('node:-', node['index'])
    print('node value:-', node['value'])
    if row[node['index']] < node['value']:  
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
 
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):  #ratio = 1.0 given
    sample = list()
    n_sample = round(len(dataset) * ratio)   # len of dataset = 208
    while len(sample) < n_sample:           #  len(sample) < 208 till than
        index = randrange(len(dataset))     # generate the random index value in range 0 - 208
        sample.append(dataset[index])       #  add the index row values to sample => subsample
    return sample
 
# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)
 
# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)  # return root node 
        trees.append(tree)
    #print('tree', tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return trees, predictions



In [28]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [29]:
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    print(unique)
    for i, value in enumerate(unique):
        #print(value)
        #print(i)
        lookup[value] = i
        #print('for value', value,'-',lookup[value])
    for row in dataset:
        #print(row)
        row[column] = lookup[row[column]]
    return lookup

In [30]:
# split the alpha string to numeric data
def str_char_to_num(dataset,column):
    for row in dataset:
        #print('rowcolumn', type(row[column]))
        i = list((str(row[column]).split())[0])[0]
        row[column] = i
    '''#print('n',i) 
    if i == 'm':
        row[2] = 0
    else:
        row[2] = 1
    j = list((row[6].split())[0])[0]
    row[6] = j
    k = list((row[8].split())[0])[0]
    row[8] = k'''
    

In [31]:
filename = 'train.csv'
dataset = load_csv(filename)
df = pd.read_csv("train.csv",header=0)
cols = df.columns.tolist()
df = df[['PassengerId', 'Pclass','Sex', 'Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked', 'Survived']]
df.fillna(0,inplace=True)
print(type(df))
print(type(dataset))
data=df.values.tolist()
print(type(data))

<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'list'>


In [32]:
for row in data:
    print('row', row)
    break
    
str_char_to_num(data, len(data[0])-2) 
str_char_to_num(data, len(data[0])-3) 
str_char_to_num(data, len(data[0])-5) 
str_char_to_num(data, len(data[0])-9)
str_column_to_int(data, len(data[0])-2) 
str_column_to_int(data, len(data[0])-3) 
str_column_to_int(data, len(data[0])-5) 
str_column_to_int(data, len(data[0])-9) 
for row in data:
    print('row', row)
    break


row [1, 3, 'male', 22.0, 1, 0, 'A/5 21171', 7.25, 0, 'S', 0]
{'S', 'C', '0', 'Q'}
{'0', 'B', 'T', 'G', 'E', 'D', 'A', 'C', 'F'}
{'8', 'W', '5', '2', '7', '6', '3', 'L', 'S', 'A', '9', '1', 'C', '4', 'F', 'P'}
{'f', 'm'}
row [1, 3, 1, 22.0, 1, 0, 9, 7.25, 0, 0, 0]


In [33]:
seed(2)
#evaluate algorithm
n_folds = 10
max_depth = 7
min_size = 1
sample_size = 0.95
n_features = int(sqrt(len(dataset[0])-1))
#print(n_features)
for n_trees in [10]:
    tree, scores = evaluate_algorithm(data, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    #print('tree',tree)
    

TypeError: get_split() takes 1 positional argument but 2 were given

In [24]:
filename = 'test.csv'
testset = load_csv(filename)
df = pd.read_csv("test.csv",header=0)
cols = df.columns.tolist()
df = df[['PassengerId', 'Pclass', 'Sex', 'Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']]
df.fillna(0,inplace=True)
print(type(df))
print(type(testset))
test=df.values.tolist()
print(type(test))
for row in test:
    print('row', row)
    break

<class 'pandas.core.frame.DataFrame'>
<class 'list'>
<class 'list'>
row [892, 3, 'male', 34.5, 0, 0, '330911', 7.8292, 0, 'Q']


In [26]:
for row in test:
    print('row', row)
    break
    
str_char_to_num(test, len(test[0])-1) 
str_char_to_num(test, len(test[0])-2) 
str_char_to_num(test, len(test[0])-4) 
str_char_to_num(test, len(test[0])-8) 
str_column_to_int(test, len(test[0])-1) 
str_column_to_int(test, len(test[0])-2) 
str_column_to_int(test, len(test[0])-4) 
str_column_to_int(test, len(test[0])-8)

for row in test:
    print('row', row)
    
import csv
sub_data=[]
schema = ['PassengerId','Survived']
with open('submission.csv', 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow([g for g in schema]) 
    for row in test:
        print('row',row[0])
        test_predict = bagging_predict(tree, row)
        #print('predict', test_predict)
        sub_data = list([row[0], test_predict])
        print(sub_data)
        writer.writerow(sub_data)
f.close()
sub = pd.read_csv("submission.csv")
print(len(sub))        

row [892, 3, 'm', 34.5, 0, 0, 3, 7.8292, 0, 2]
{'2', '0', '1'}
{'0', '7', '2', '6', '3', '1', '4', '5'}
{'8', '0', '2', '7', '6', '3', '9', '1', '4', '5'}
{'f', 'm'}
row [892, 3, 1, 34.5, 0, 0, 5, 7.8292, 0, 0]
row [893, 3, 0, 47.0, 1, 0, 5, 7.0, 0, 1]
row [894, 2, 1, 62.0, 0, 0, 1, 9.6875, 0, 0]
row [895, 3, 1, 27.0, 0, 0, 5, 8.6625, 0, 1]
row [896, 3, 0, 22.0, 1, 1, 5, 12.2875, 0, 1]
row [897, 3, 1, 14.0, 0, 0, 7, 9.225, 0, 1]
row [898, 3, 0, 30.0, 0, 0, 5, 7.6292, 0, 0]
row [899, 2, 1, 26.0, 1, 1, 1, 29.0, 0, 1]
row [900, 3, 0, 18.0, 0, 0, 1, 7.2292, 0, 2]
row [901, 3, 1, 21.0, 2, 0, 4, 24.15, 0, 1]
row [902, 3, 1, 0.0, 0, 0, 5, 7.8958, 0, 1]
row [903, 1, 1, 46.0, 0, 0, 2, 26.0, 0, 1]
row [904, 1, 0, 23.0, 1, 0, 1, 82.2667, 5, 1]
row [905, 2, 1, 63.0, 1, 0, 1, 26.0, 0, 1]
row [906, 1, 0, 47.0, 1, 0, 6, 61.175, 4, 1]
row [907, 2, 0, 24.0, 1, 0, 9, 27.7208, 0, 2]
row [908, 2, 1, 35.0, 0, 0, 1, 12.35, 0, 0]
row [909, 3, 1, 21.0, 0, 0, 1, 7.225, 0, 2]
row [910, 3, 0, 27.0, 1, 0, 9, 7.92

node:- 2
node value:- 1
node:- 0
node value:- 397
node:- 1
node value:- 3
node:- 9
node value:- 3
node:- 8
node value:- 2
node:- 7
node value:- 10.5
node:- 0
node value:- 379
node:- 7
node value:- 7.875
node:- 0
node value:- 511
node:- 2
node value:- 1
node:- 3
node value:- 30.5
node:- 7
node value:- 10.5
node:- 3
node value:- 47.0
node:- 3
node value:- 45.0
node:- 3
node value:- 33.0
node:- 3
node value:- 15.0
node:- 9
node value:- 2
node:- 6
node value:- 5
[900, 0]
row 901
node:- 2
node value:- 1
node:- 8
node value:- 2
node:- 0
node value:- 716
node:- 7
node value:- 56.4958
node:- 5
node value:- 1
node:- 0
node value:- 823
node:- 1
node value:- 3
node:- 6
node value:- 5
node:- 4
node value:- 2
node:- 2
node value:- 1
node:- 5
node value:- 2
node:- 3
node value:- 7.0
node:- 9
node value:- 1
node:- 2
node value:- 1
node:- 1
node value:- 2
node:- 8
node value:- 7
node:- 0
node value:- 829
node:- 9
node value:- 2
node:- 7
node value:- 11.1333
node:- 1
node value:- 3
node:- 8
node value:

node value:- 1
node:- 2
node value:- 1
node:- 0
node value:- 397
node:- 1
node value:- 3
node:- 9
node value:- 3
node:- 8
node value:- 2
node:- 7
node value:- 10.5
node:- 0
node value:- 379
node:- 7
node value:- 7.875
node:- 0
node value:- 511
node:- 2
node value:- 1
node:- 3
node value:- 30.5
node:- 7
node value:- 10.5
node:- 3
node value:- 47.0
node:- 3
node value:- 45.0
node:- 5
node value:- 0
[911, 0]
row 912
node:- 2
node value:- 1
node:- 8
node value:- 2
node:- 0
node value:- 716
node:- 7
node value:- 56.4958
node:- 6
node value:- 7
node:- 6
node value:- 5
node:- 1
node value:- 3
node:- 7
node value:- 12.35
node:- 8
node value:- 6
node:- 7
node value:- 65.0
node:- 5
node value:- 1
node:- 3
node value:- 24.0
node:- 2
node value:- 1
node:- 1
node value:- 2
node:- 8
node value:- 4
node:- 4
node value:- 1
node:- 0
node value:- 225
node:- 7
node value:- 66.6
node:- 6
node value:- 7
node:- 8
node value:- 2
node:- 1
node value:- 3
node:- 7
node value:- 18.75
node:- 6
node value:- 5
node

node:- 5
node value:- 1
node:- 2
node value:- 1
node:- 4
node value:- 2
node:- 6
node value:- 8
node:- 5
node value:- 0
node:- 8
node value:- 2
node:- 7
node value:- 10.5
node:- 2
node value:- 1
node:- 0
node value:- 66
node:- 8
node value:- 1
node:- 4
node value:- 1
node:- 7
node value:- 10.5
node:- 2
node value:- 1
node:- 8
node value:- 2
node:- 5
node value:- 1
node:- 1
node value:- 2
node:- 7
node value:- 11.5
node:- 1
node value:- 3
[922, 0]
row 923
node:- 2
node value:- 1
node:- 8
node value:- 2
node:- 0
node value:- 716
node:- 7
node value:- 56.4958
node:- 5
node value:- 1
node:- 0
node value:- 823
node:- 1
node value:- 3
node:- 6
node value:- 5
node:- 1
node value:- 3
node:- 7
node value:- 12.35
node:- 8
node value:- 6
node:- 7
node value:- 65.0
node:- 5
node value:- 1
node:- 3
node value:- 24.0
node:- 2
node value:- 1
node:- 1
node value:- 2
node:- 8
node value:- 7
node:- 0
node value:- 829
node:- 9
node value:- 2
node:- 7
node value:- 11.1333
node:- 1
node value:- 3
node:- 8


node:- 7
node value:- 18.75
node:- 2
node value:- 1
node:- 8
node value:- 1
node:- 5
node value:- 0
node:- 1
node value:- 3
node:- 8
node value:- 2
node:- 2
node value:- 1
node:- 7
node value:- 26.25
node:- 0
node value:- 54
node:- 3
node value:- 38.0
node:- 4
node value:- 1
node:- 2
node value:- 1
node:- 7
node value:- 49.5
node:- 5
node value:- 2
node:- 3
node value:- 44.0
node:- 4
node value:- 3
node:- 0
node value:- 114
node:- 7
node value:- 11.1333
node:- 7
node value:- 11.1333
node:- 2
node value:- 1
node:- 4
node value:- 3
node:- 1
node value:- 3
node:- 3
node value:- 24.0
node:- 5
node value:- 2
node:- 3
node value:- 28.0
node:- 7
node value:- 56.4958
node:- 8
node value:- 2
node:- 5
node value:- 1
node:- 2
node value:- 1
node:- 0
node value:- 397
node:- 1
node value:- 3
node:- 8
node value:- 1
node:- 8
node value:- 2
node:- 7
node value:- 10.5
node:- 2
node value:- 1
node:- 6
node value:- 6
node:- 0
node value:- 886
node:- 7
node value:- 10.5
node:- 2
node value:- 1
node:- 7
n

...

node value:- 33.0
node:- 3
node value:- 15.0
node:- 0
node value:- 122
node:- 7
node value:- 7.8292
[1249, 0]
row 1250
node:- 2
node value:- 1
node:- 8
node value:- 2
node:- 0
node value:- 716
node:- 7
node value:- 56.4958
node:- 5
node value:- 1
node:- 0
node value:- 823
node:- 1
node value:- 3
node:- 6
node value:- 5
node:- 1
node value:- 3
node:- 5
node value:- 2
node:- 2
node value:- 1
node:- 5
node value:- 1
node:- 7
node value:- 56.4958
node:- 9
node value:- 3
node:- 2
node value:- 1
node:- 1
node value:- 2
node:- 8
node value:- 7
node:- 0
node value:- 829
node:- 9
node value:- 2
node:- 7
node value:- 11.1333
node:- 2
node value:- 1
node:- 8
node value:- 2
node:- 1
node value:- 3
node:- 3
node value:- 36.0
node:- 0
node value:- 321
node:- 9
node value:- 2
node:- 7
node value:- 16.1
node:- 5
node value:- 1
node:- 1
node value:- 3
node:- 8
node value:- 7
node:- 9
node value:- 3
node:- 8
node value:- 5
node:- 3
node value:- 34.0
node:- 4
node value:- 3
node:- 6
node value:- 7
node:-

node value:- 1
node:- 0
node value:- 66
node:- 8
node value:- 1
node:- 4
node value:- 1
node:- 7
node value:- 10.5
node:- 2
node value:- 1
node:- 8
node value:- 2
node:- 5
node value:- 1
node:- 1
node value:- 2
node:- 7
node value:- 11.5
node:- 1
node value:- 3
[1262, 0]
row 1263
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 8
node value:- 6
node:- 4
node value:- 1
node:- 6
node value:- 7
node:- 0
node value:- 248
node:- 5
node value:- 2
node:- 6
node value:- 5
node:- 4
node value:- 2
node:- 0
node value:- 46
node:- 2
node value:- 1
node:- 5
node value:- 2
node:- 7
node value:- 11.1333
node:- 0
node value:- 80
node:- 2
node value:- 1
node:- 6
node value:- 5
node:- 5
node value:- 2
node:- 1
node value:- 3
node:- 5
node value:- 0
node:- 8
node value:- 2
node:- 0
node value:- 299
node:- 2
node value:- 1
node:- 3
node value:- 57.0
node:- 5
node value:- 1
node:- 4
node value:- 0
node:- 1
node value:- 3
node:- 8
node value:- 2
node:- 3
node value:- 54.0
node:- 3
node value:- 48.0
no

node value:- 397
node:- 1
node value:- 3
node:- 9
node value:- 3
node:- 8
node value:- 2
node:- 7
node value:- 10.5
node:- 2
node value:- 1
node:- 6
node value:- 6
node:- 0
node value:- 886
node:- 7
node value:- 10.5
node:- 2
node value:- 1
node:- 7
node value:- 49.5042
node:- 4
node value:- 3
node:- 1
node value:- 3
node:- 3
node value:- 9.0
node:- 0
node value:- 853
[1275, 0]
row 1276
node:- 2
node value:- 1
node:- 8
node value:- 2
node:- 0
node value:- 716
node:- 7
node value:- 56.4958
node:- 5
node value:- 1
node:- 0
node value:- 823
node:- 1
node value:- 3
node:- 6
node value:- 5
node:- 1
node value:- 3
node:- 7
node value:- 12.35
node:- 8
node value:- 6
node:- 7
node value:- 65.0
node:- 5
node value:- 1
node:- 3
node value:- 24.0
node:- 2
node value:- 1
node:- 1
node value:- 2
node:- 8
node value:- 7
node:- 0
node value:- 829
node:- 9
node value:- 2
node:- 7
node value:- 11.1333
node:- 1
node value:- 3
node:- 8
node value:- 2
node:- 1
node value:- 3
node:- 7
node value:- 18.75
no

node:- 7
node value:- 11.1333
node:- 6
node value:- 7
node:- 9
node value:- 2
node:- 0
node value:- 13
node:- 0
node value:- 107
node:- 7
node value:- 8.3
node:- 0
node value:- 223
node:- 7
node value:- 56.4958
node:- 8
node value:- 2
node:- 5
node value:- 1
node:- 2
node value:- 1
node:- 4
node value:- 2
node:- 6
node value:- 8
node:- 6
node value:- 1
node:- 8
node value:- 2
node:- 7
node value:- 10.5
node:- 0
node value:- 379
node:- 7
node value:- 7.875
node:- 0
node value:- 511
node:- 2
node value:- 1
node:- 0
node value:- 569
node:- 7
node value:- 10.5
node:- 3
node value:- 47.0
node:- 3
node value:- 45.0
node:- 3
node value:- 33.0
node:- 3
node value:- 15.0
node:- 9
node value:- 2
node:- 8
node value:- 7
[1288, 0]
row 1289
node:- 2
node value:- 1
node:- 1
node value:- 3
node:- 8
node value:- 6
node:- 4
node value:- 1
node:- 7
node value:- 151.55
node:- 0
node value:- 317
node:- 7
node value:- 27.0
node:- 6
node value:- 5
node:- 4
node value:- 2
node:- 0
node value:- 46
node:- 2
no

node:- 5
node value:- 1
node:- 1
node value:- 3
node:- 8
node value:- 7
node:- 9
node value:- 3
node:- 8
node value:- 5
node:- 3
node value:- 34.0
node:- 4
node value:- 3
node:- 6
node value:- 7
node:- 2
node value:- 1
node:- 7
node value:- 49.5
node:- 5
node value:- 2
node:- 3
node value:- 44.0
node:- 4
node value:- 3
node:- 0
node value:- 114
node:- 7
node value:- 11.1333
node:- 7
node value:- 11.1333
node:- 2
node value:- 1
node:- 4
node value:- 3
node:- 1
node value:- 3
node:- 5
node value:- 2
node:- 3
node value:- 45.0
node:- 3
node value:- 18.0
node:- 7
node value:- 56.4958
node:- 8
node value:- 2
node:- 5
node value:- 1
node:- 4
node value:- 3
node:- 0
node value:- 710
node:- 3
node value:- 28.0
node:- 6
node value:- 7
node:- 8
node value:- 2
node:- 7
node value:- 10.5
node:- 2
node value:- 1
node:- 6
node value:- 6
node:- 0
node value:- 57
node:- 3
node value:- 21.0
node:- 7
node value:- 26.25
node:- 7
node value:- 10.5
node:- 2
node value:- 1
node:- 7
node value:- 49.5042
node